# Atividade Prática 7

## Família de Wavelets de Haar e Representações Matriciais

### Entrega: até 27/06/2020 às 23:59 no e-disciplinas

#### Nome: ________________________ N° USP: ________  ( ) Grad ( ) Pós

---

In [ ]:
# importa dependências
import math as m
import numpy as np
from scipy.linalg import circulant
import matplotlib.pyplot as plt
from ipywidgets import Dropdown, interactive

---

### Revisão: Formas básicas de onda associadas a uma transformada genérica

Como vimos no caso da DFT e da DCT em $\mathbb{C}^N$, essas transformadas possuem uma representação matricial de tal forma que a equação de análise (transformada direta) sempre pode ser representada por uma equação

$$X = \mathcal{A}x$$

para alguma matriz $\mathcal{A}\in\mathcal{M}_{N\times N}(\mathbb{C})$ adequada. Para a transformada de Fourier, por exemplo, vimos na seção 2.5.1 que $\mathcal{A} = \mathcal{F}_N$ com componentes

$$\left(\mathcal{F}_N\right)_{k,n} = e^{-i2\pi k n/N},$$

ao passo que para a DCT vimos na seção 3.4.3 que $\mathcal{A} = \mathcal{C}_N$ com componentes

$$\begin{equation*}
	(\mathcal{C}_N)_{k,m} = \left\{\begin{array}{ll}
		\sqrt{\frac{1}{N}} & \text{ se }k = 0 \\
		\sqrt{\frac{2}{N}}\cos\left(\frac{\pi k\left(m + \frac{1}{2}\right)}{N}\right) & \text{ se } k = 1, 2, \dots, N-1.
	\end{array}\right.
\end{equation*}$$

Em qualquer um dos casos, a equação de síntese (transformada inversa) era sempre dada por

$$x = \mathcal{A}^{-1}X = \sum_{k=0}^{N-1}X_kV_k$$

onde $V_k$ é a k-ésima coluna de $\mathcal{A}^{-1}$, ou seja

$$V_k = \mathcal{A}^{-1}\varepsilon_k,$$

onde $\varepsilon_k=(0,\ldots,0,\overbrace{1}^k,0,\ldots,0)\in\mathbb{C}^N$ é o $k$-ésimo vetor da base canônica. Na DFT, tínhamos $V_k=E_k=\text{IDFT}(\varepsilon_k)$, a $k$-ésima forma básica de onda de Fourier correspondendo a uma exponencial complexa de frequência $k$ (ciclos por $N$ amostras) amostrada nos instantes $n=0,1,\ldots,N-1$, enquanto na DCT tínhamos $V_k=\mathcal{C}_k=\text{IDCT}(\varepsilon_k)$, uma função cossenoidal de frequência $\frac{k}{2}$ (ciclos por $N$ amostras), amostrada em instantes da forma $m+\frac{1}{2}$ para $m=0,1,\ldots,N-1$.

### E as formas básicas de onda das transformadas Wavelet?

Devemos lembrar que existem inúmeras transformadas Wavelet, uma para cada banco de filtros da forma $\{\ell_a,h_a,\ell_s,h_s\}$ que satisfaz a propriedade de reconstrução perfeita.

No caso do banco de filtros ortogonal de Haar

$$\begin{array}{cc}
\ell_a = (\ldots,0,\overbrace{\frac{\sqrt{2}}{2}}^{n=0},\frac{\sqrt{2}}{2},0,\ldots) &
h_a = (\ldots,0,\overbrace{\frac{\sqrt{2}}{2}}^{n=0},-\frac{\sqrt{2}}{2},0,\ldots)\\
\ell_s = (\ldots,0,\frac{\sqrt{2}}{2},\overbrace{\frac{\sqrt{2}}{2}}^{n=0},0,\ldots) &
h_s = (\ldots,0,-\frac{\sqrt{2}}{2},\overbrace{\frac{\sqrt{2}}{2}}^{n=0},0,\ldots)
\end{array}$$

as formas básicas de onda podem ser obtidas como as colunas da inversa da matriz $\mathcal{W}_N^M$, que representa a transformada Wavelet do banco de filtros ortogonal de Haar de $M$ estágios em $\mathbb{C}^N$, definida pela aplicação iterada $M$ vezes da transformação

$$x\mapsto(X_l,X_h)\mapsto(X_{ll},X_{lh},X_h)\mapsto(X_{lll},X_{llh},X_{lh},X_h)\mapsto\cdots$$

onde no $(m+1)$-ésimo estágio o segmento inicial $y=X_{\underbrace{l\ldots l}_m}\in\mathbb{C}^{\frac{N}{2^m}}$ é transformado de acordo com as equações

$$y\longrightarrow\begin{array}{|l}
\longrightarrow X_{\underbrace{l\ldots l}_ml} = D(\ell_a*y)\quad\quad\in\mathcal{C}^{\frac{N}{2^{m+1}}},\\
\longrightarrow X_{\underbrace{l\ldots l}_mh} = D(h_a*y)\quad\quad\in\mathcal{C}^{\frac{N}{2^{m+1}}},\end{array}$$

sendo que todas as convoluções são *circulares*, e as subamostragens selecionam apenas os índices pares, reduzindo à metade o tamanho dos vetores.

Antes de implementarmos qualquer coisa, vamos olhar uma implementação que já existe, a fim de ganharmos intuição sobre as propriedades da família de Wavelets de Haar

$$\big\{W_k\big\}_{k=0}^{N-1}\quad\quad\text{onde}\quad\quad W_k = \left(\mathcal{W}_N^M\right)^{-1}\varepsilon_k=\text{IDWT}_N^M(\varepsilon_k).$$

---

Leia os códigos a seguir, que implementam a DWT e a IDWT de $M$ estágios em $\mathbb{C}^N$, e explore a interface interativa que permite visualizar as formas básicas de onda para diversos valores de $M$ e $N$. Em seguida, responda às perguntas das caixas de texto.

In [ ]:
from numpy.fft import fft,ifft

# convolução circular
def cconv(x,h):    # considera que |h|<=|x|
    hpad = np.zeros(len(x))
    hpad[:len(h)] = h
    return np.real(ifft(fft(x)*fft(hpad)))

# DWT ortogonal de Haar com M estágios
# Obs: essa mesma implementação serve para outros bancos de filtros, sendo que, se 
# forem causais, apenas as duas linhas que definem la e ha precisam ser alteradas.
# Se os filtros não forem causais, será necessário fazer um shift circular das saídas
# das convoluções usando a função np.roll(..., -L), onde L é o número de coeficientes
# do filtro com índices negativos.
def dwt_haar(x, M=1):
    # filtros de análise de Haar (definidos para n=0,1)
    c = m.sqrt(2)/2
    la = np.array([c, c]) # filtro passa-baixas
    ha = np.array([c, -c]) # filtro passa-altas
    # inicializa vetor X de saída com uma cópia de x
    N = len(x)
    X = np.ndarray(np.shape(x))
    X[:] = x
    # em cada estágio j, codificaremos o vetor
    # y = X_{ll...l} = coefs de aproximação do estágio anterior
    for j in range(M):
        y = X.copy()[:N] # seleciona o trecho a codificar
        X[:N//2] = cconv(y,la)[::2] # Xl = D(x*l)
        X[N//2:N] = cconv(y,ha)[::2] # Xh = D(x*h)
        N = N//2 # próximo estágio processará metade do vetor
    return X

# IDWT ortogonal de Haar com M estágios
# Obs: essa mesma implementação serve para outros bancos de filtros. Para o banco
# de filtros de Haar, cada filtro de síntese possui L=1 coeficientes com índices
# negativos, e por isso as saídas das convoluções são corrigidas por um shift
# circular usando a função np.roll(..., -L).
def idwt_haar(X, M=1):
    # filtros de síntese de Haar (definidos para n=-1,0)
    c = m.sqrt(2)/2
    ls = np.array([ c, c ]) # filtro passa-baixas
    hs = np.array([ -c, c ]) # filtro passa-altas
    # inicializa vetor x da saída
    x = X.copy()
    # começa a reconstrução do último nível
    n = len(X)//2**M
    for j in range(M,0,-1):
        # coeficientes de aproximação e detalhes do nível j
        cA, cD = x[:n], x[n:2*n]
        # superamostragem
        UXl = np.zeros(2*n); UXl[::2] = cA
        UXh = np.zeros(2*n); UXh[::2] = cD
        # filtragem (a função np.roll ajusta o shift dos filtros de síntese)
        vl, vh = np.roll(cconv(UXl, ls), -1), np.roll(cconv(UXh, hs), -1)
        x[:2*n] = vl+vh # combina canais
        n = n*2 # próximo nível terá o dobro dos coeficientes
    return x

#### A interface a seguir plota a família de funções $\{W_k\}$ para $N$ e $M$ escolhidos

In [ ]:
# Dimensão do espaço
N = Dropdown(options=[4,8,16,32], value=4, 
             description='Dimensão: ', style={'description_width': 'initial'})

# Número de estágios da DWT
M = Dropdown(options=[1,2], value=1, 
             description='Estágios: ', style={'description_width': 'initial'})

# Redefine o menu Dropdown do M cada vez que N é alterado
def updateM(x):
    global N
    global M
    M.index = 0
    M.options = range(1,int(m.log2(N.value)+1))

# Opção para usar a função definida no exercício 2
Exercicio = Dropdown(options=[True, False], value=False, 
             description='Minha solução: ', style={'description_width': 'initial'})

def plota_familia(N,M,Exercicio):
    # testa se é possível ir até o M-ésimo estágio
    # (apenas para chamadas diretas da função, pois a interface abaixo não permite)
    if 2**M>N:
        M = int(m.log2(N))
        print(f"Usando estágio máximo M={M} para N={N}")
    # testa se Exercício=True sem a função estar implementada
    if Exercicio and "matrizDWT" not in dir():
        print("Você ainda não implementou a função do exercício 2.")
        print("Ignorando a opção Minha Solução=True.")
        Exercicio = False
    # configura dimensões das figuras
    L = int(m.sqrt(N))
    A = N//L if N%L==0 else N//L+1
    fig, ax = plt.subplots(A,L,figsize=(15, 5*A))
    # cria vetor da base canônica para gerar formas básicas de onda pela IDWT
    X = np.zeros(N)
    index = 0,0 # usado para posicionar a figura
    for i in range(N):
        X[i] = 1; # marca i-ésimo vetor da base canônica
        if not Exercicio: # essa é a implementação dada
            x = idwt_haar(X,M=M)
        else: # essa é a implementação que você deverá fazer
            x = np.linalg.inv(matrizDWT(N,M)).dot(X)
        X[i] = 0 # restaura vetor para próxima iteração
        # formata a figura para evitar a interpolação linear dos coeficientes:
        # cada valor X_k é "mantido" até quase o índice k+1 (sample-and-hold)
        x = np.append(x,x[N-1])
        ax[index].step(range(N+1),x,where='post')
        ax[index].set_title(f'W[{i}]')
        ax[index].axis([0,N,-1,1])
        # atualiza posição da figura
        if index[1]<L-1:
            index = index[0],index[1]+1
        else:
            index = index[0]+1,0
    # limpa últimos gráficos vazios
    for i in range(N,L*A):
        ax[index].axis('off')
        # atualiza posição da figura
        if index[1]<L-1:
            index = index[0],index[1]+1
        else:
            index = index[0]+1,0
    plt.show()

# configura as opções do número de estágios cada vez que a dimensão varia

N.observe(updateM)

# interface gráfica interativa para gerar formas básicas de onda da DWT de Haar
interactive(plota_familia, N=N, M=M, Exercicio=Exercicio)

---

#### Exercício 1: Responda as perguntas abaixo.

**a)** Comente as propriedades que você observa nas formas de onda usando os valores $N=4,8,16,32$ e a transformada de $M=1$ estágio. Elas parecem ter alguma relação com os coeficientes dos filtros $\ell_a$ e $h_a$? Quais são as diferenças e semelhanças entre as formas $W_k$ para $k<\frac{N}{2}$ e para $k\ge\frac{N}{2}$?

##### Resposta do exercício 1a:

##### begin gabarito

As formas básicas de onda para $k<\frac{N}{2}$ parecem refletir as ações do filtro passa-baixas $\ell_a$, que combina linearmente duas amostras sucessivas, sendo que cada forma de onda $W_k$ possui pesos iguais em dois coeficientes sucessivos, pulando de 2 em 2, ou seja, $W_0$ possui peso não-nulo em $x_0$ e $x_{-1}$ (que é $=x_{N-1}$), $W_1$ possui peso não-nulo em $x_2$ e $x_1$, e assim por diante. A partir de $k\ge\frac{N}{2}$ as formas parecem se relacionar com o filtro passa-altas $h_a$, pois possuem pesos opostos em dois coeficientes sucessivos, onde $W_{N/2}$ tem pesos em $+x_0$ e $-x_{-1}$, $W_{N/2+1}$ em $+x_2$ e $-x_1$, e assim por diante.

##### end gabarito

**b)** Considerando os valores mais altos ($N=16,32$), varie o número de estágios $M$ da transformada e observe quais formas de onda são alteradas. Você consegue detectar algum padrão? Há formas que não mudam de $M=1$ para $M=2$? E de $M=2$ para $M=3$? No caso das formas que mudam quando $M$ aumenta, que tipo de perfil elas tinham antes ($M$ menor) e depois ($M$ maior)? Há alguma relação geométrica notável entre as formas básicas de onda através dos diversos estágios da transformada?

##### Resposta do exercício 1b:

##### begin gabarito

Usando $N=16,32$ e variando $M=1$ para $M=2$, é possível notar que as formas para $k\ge\frac{N}{2}$ *não* mudam, mas aquelas para $k<\frac{N}{2}$ são alteradas e dão origem a dois padrões diferentes: para $k<\frac{N}{4}$ as formas têm o aspecto "passa-baixas", porém cobrindo 4 amostras seguidas ($W_0$ cobre as amostras $x_0,x_{-1},x_{-2},x_{-3}$, $W_1$ cobre $x_4,x_3,x_2,x_1$, e assim por diante), enquanto para $\frac{N}{4}\le k<\frac{N}{2}$ elas têm o aspecto "passa-altas", porém considerando diferenças entre blocos de duas amostras ($W_{\frac{N}{4}}$ tem pesos $+x_0,+x_{-1},-x_{-2},-x_{-3}$, $W_{\frac{N}{4}+1}$ tem pesos $+x_4,+x_3,-x_2,-x_1$, e assim por diante).

Ao variar de $M=2$ para $M=3$, todas as formas para $k\ge\frac{N}{4}$ *não* mudam, mas aquelas para $k<\frac{N}{4}$ se dividem em dois grupos, um primeiro que corresponde a pesos iguais em sequências de 8 amostras, e outro que corresponde pesos de sinal diferente entre grupos de 4 amostras consecutivos. Todas as formas para $k\ge\frac{N}{8}$ têm uma geometria semelhante, com um padrão de sinal alternado que muda de largura conforme o estágio em que a forma foi "criada": as primeiras formas (estágio 1) são as mais "estreitas" (uma amostra positiva e uma negativa), as do 2° estágio têm largura dobrada, as do 3° têm largura quadruplicada, e assim por diante.

##### end gabarito

---

### Representação matricial da DWT em 1 estágio

Vimos que a transformada em 1 estágio, que corresponde à transformação

$$x\mapsto(X_l,X_h)=\begin{bmatrix}D(\ell_a*x)\\D(h_a*x)\end{bmatrix},$$

pode ser representada matricialmente se considerarmos a representação matricial da convolução

$$y*x = M_yx$$

onde $M_y\in\mathcal{M}_{N\times N}(\mathbb{C})$ é a matriz circulante associada a $y$, cujas entradas são $(M_y)_{i,j}=y_{i-j\ \%\ N}$, e também a representação matricial da subamostragem

$$D(z) = \begin{bmatrix}z_0\\z_2\\\vdots\\z_{N-2}\end{bmatrix} = 
\begin{bmatrix}
1&0&0&0&0&0&\cdots&0\\
0&0&1&0&0&0&\cdots&0\\
0&0&0&0&1&0&\cdots&0\\
\vdots&&&&&&\ddots&\vdots\\
0&0&0&0&0&0&\cdots&1
\end{bmatrix}\begin{bmatrix}z_0\\z_1\\\vdots\\z_{N-1}\end{bmatrix},$$

onde $D\in\mathcal{M}_{\frac{N}{2}\times N}(\mathbb{C})$ é a matriz tal que $D_{i,2i}=1$ e $D_{i,j}=0$ caso contrário.

Assim,

$$x\mapsto(X_l,X_h)=\begin{bmatrix}D(\ell_a*x)\\D(h_a*x)\end{bmatrix}
=\begin{bmatrix}DM_{\ell_a}x\\DM_{h_a}x\end{bmatrix}
=\begin{bmatrix}L_N^1\\H_N^1\end{bmatrix}x = \mathcal{W}_N^Mx$$

onde $\mathcal{W}_N^1=\begin{bmatrix}L_N^1\\H_N^1\end{bmatrix}$, sendo que $L_N^1\in\mathcal{M}_{\frac{N}{2}\times N}(\mathbb{C})$ é a matriz formada *apenas pelas linhas pares* da matriz circulante $M_{\ell_a}$, e $H_N^1\in\mathcal{M}_{\frac{N}{2}\times N}(\mathbb{C})$ é a matriz formada apenas pelas linhas pares da matriz circulante $M_{h_a}$.

### Representação matricial da DWT em $M$ estágios

Lembrando que a DWT em vários estágios *conserva todos os coeficientes de detalhes de estágios anteriores*, e a cada passo apenas recodifica os coeficientes de aproximação do último estágio, que correspondem aos primeiros $\frac{N}{2^m}$ coeficientes no estágio $m+1$, vemos que a estrutura da matriz que representa essa recodificação só é alterada nas suas primeiras $\frac{N}{2^m}$ linhas. Assim, para passar da DWT de 1 estágio para a de 2 estágios, é necessário considerar as matrizes que representam as DWTs de 1 estágio em $\mathbb{C}^N$ e $\mathbb{C}^{\frac{N}{2}}$

$$\mathcal{W}_N^1=\begin{bmatrix}L_N^1\\H_N^1\end{bmatrix}\in\mathcal{M}_{N\times N}(\mathbb{C})\quad\quad\text{e}\quad\quad\mathcal{W}_{N/2}^1=\begin{bmatrix}L_{N/2}^1\\H_{N/2}^1\end{bmatrix}\in\mathcal{M}_{\frac{N}{2}\times\frac{N}{2}}(\mathbb{C})$$

e construir a matriz

$$\mathcal{W}_N^2=\begin{bmatrix}\mathcal{W}_{N/2}^1L_N^1\\H_N^1\end{bmatrix}
= \begin{bmatrix}L_N^2\\H_N^2\\H_N^1\end{bmatrix}$$

onde $L_N^2$ e $H_N^2$ são respectivamente as primeiras $\frac{N}{4}$ e as últimas $\frac{N}{4}$ linhas de $\mathcal{W}_{N/2}^1L_N^1\in\mathcal{M}_{\frac{N}{2}\times N}(\mathbb{C})$.

Para generalizar esse processo, devemos então a cada estágio $m=1,2,\ldots,M$ substituir o trecho inicial $L_N^m\in\mathcal{M}_{\frac{N}{2^m}\times N}$ da matriz

$$\mathcal{W}_N^m = \begin{bmatrix}L_N^m\\H_N^{m}\\H_N^{m-1}\\\vdots\\H_N^1\end{bmatrix}$$ 

por sua recodificação
$$\mathcal{W}_{N/2^m}^1L_N^m = \begin{bmatrix}L_N^{m+1}\\H_N^{m+1}\end{bmatrix},$$

obtendo assim a matriz que representa a DWT no estágio $m+1$, dada por

$$\mathcal{W}_N^{m+1} = \begin{bmatrix}L_N^{m+1}\\H_N^{m+1}\\H_N^{m}\\H_N^{m-1}\\\vdots\\H_N^1\end{bmatrix}$$ 


---

#### Exercício 2

Implemente uma função <tt>matrizDWT(N,M)</tt> que devolva a matriz $N\times N$ que represente a DWT de $M$ estágios em $\mathbb{C}^N$. Volte à interface interativa anterior e verifique que ela continua produzindo as mesmas matrizes com a opção <tt>"Exercício=True"</tt>.

**Dicas:** *Use a função <tt>circulant</tt> (já importada) sobre os vetores de coeficientes do filtro completados com zeros em $\mathbb{C}^N$, e preocupe-se inicialmente com o caso $M=1$, testando sua função com as chamadas*

<tt>print(matrizDWT(4,1))<br>
print(matrizDWT(8,1))</tt>

*Apenas depois de se certificar de que a função funciona para $M=1$, preocupe-se com o caso $M>1$. Você pode resolvê-lo tanto usando recursão (com uma única chamada recursiva de sua função) quanto de forma interativa (com um laço que faz $\log_2(N)$ chamadas recursivas). Teste o caso geral com as chamadas*

<tt>print(matrizDWT(4,2))<br>
print(matrizDWT(8,2))<br>
print(matrizDWT(8,3))</tt>

*e compare os resultados com as matrizes disponíveis no anexo ao final deste notebook*.

In [ ]:
# Resposta do exercício 2

# begin gabarito
def matrizDWT(N,M):
    # definição do banco de filtros
    c = m.sqrt(2)/2
    la = [c, c]
    ha = [c, -c]
    # preenche vetores até o tamanho N
    laN = np.zeros(N);laN[:len(la)] = la
    haN = np.zeros(N);haN[:len(ha)] = ha
    # cria matriz
    W = np.zeros((N,N))
    # preenche primeira metade com canal passa-baixas
    W[:N//2] = circulant(laN)[::2]
    # preenche segunda metade com canal passa-altas
    W[N//2:] = circulant(haN)[::2]
    if M>1:
        W_2 = matrizDWT(N//2,M-1)
        W[:N//2] = W_2.dot(W[:N//2])
    return W

for N in [4,8]:
    for M in range(1,int(m.log2(N))+1):
        print(f"W_{N}^{M} = \n",matrizDWT(N,M))

# end gabarito

### Anexo

$$\mathcal{W}_{4}^{1} = \begin{bmatrix} \frac{\sqrt{2}}{2}&0&0&\frac{\sqrt{2}}{2}\\
  0&\frac{\sqrt{2}}{2}&\frac{\sqrt{2}}{2}&0\\
\frac{\sqrt{2}}{2}&0&0&-\frac{\sqrt{2}}{2}\\
  0&-\frac{\sqrt{2}}{2}&\frac{\sqrt{2}}{2}&0&\end{bmatrix}$$
  
$$\mathcal{W}_{4}^{2} = \begin{bmatrix} \frac{1}{2}&\frac{1}{2}&\frac{1}{2}&\frac{1}{2}\\
\frac{1}{2}&-\frac{1}{2}&-\frac{1}{2}&\frac{1}{2}\\
\frac{\sqrt{2}}{2}&0&0&-\frac{\sqrt{2}}{2}\\
  0&-\frac{\sqrt{2}}{2}&\frac{\sqrt{2}}{2}&0&\end{bmatrix}$$
  
$$\mathcal{W}_{8}^{1} = \begin{bmatrix} \frac{\sqrt{2}}{2}&0&0&0&0&0
&0&\frac{\sqrt{2}}{2}\\
  0&\frac{\sqrt{2}}{2}&\frac{\sqrt{2}}{2}&0&0&0
&0&0\\
  0&0&0&\frac{\sqrt{2}}{2}&\frac{\sqrt{2}}{2}&0
&0&0\\
  0&0&0&0&0&\frac{\sqrt{2}}{2}
&\frac{\sqrt{2}}{2}&0\\
\frac{\sqrt{2}}{2}&0&0&0&0&0
&0&-\frac{\sqrt{2}}{2}\\
  0&-\frac{\sqrt{2}}{2}&\frac{\sqrt{2}}{2}&0&0&0
&0&0\\
  0&0&0&-\frac{\sqrt{2}}{2}&\frac{\sqrt{2}}{2}&0
&0&0\\
  0&0&0&0&0&-\frac{\sqrt{2}}{2}
&\frac{\sqrt{2}}{2}&0&\end{bmatrix}$$

$$\mathcal{W}_{8}^{2} = \begin{bmatrix} \frac{1}{2}&0&0&0&0&\frac{1}{2}
&\frac{1}{2}&\frac{1}{2}\\
  0&\frac{1}{2}&\frac{1}{2}&\frac{1}{2}&\frac{1}{2}&0
&0&0\\
\frac{1}{2}&0&0&0&0&-\frac{1}{2}
 &-\frac{1}{2}&\frac{1}{2}\\
  0&-\frac{1}{2}&-\frac{1}{2}&\frac{1}{2}&\frac{1}{2}&0
&0&0\\
\frac{\sqrt{2}}{2}&0&0&0&0&0
&0&-\frac{\sqrt{2}}{2}\\
  0&-\frac{\sqrt{2}}{2}&\frac{\sqrt{2}}{2}&0&0&0
&0&0\\
  0&0&0&-\frac{\sqrt{2}}{2}&\frac{\sqrt{2}}{2}&0
&0&0\\
  0&0&0&0&0&-\frac{\sqrt{2}}{2}
&\frac{\sqrt{2}}{2}&0&\end{bmatrix}$$

$$\mathcal{W}_{8}^{3} = \begin{bmatrix} \frac{\sqrt{2}}{4}&\frac{\sqrt{2}}{4}&\frac{\sqrt{2}}{4}&\frac{\sqrt{2}}{4}&\frac{\sqrt{2}}{4}&\frac{\sqrt{2}}{4}
&\frac{\sqrt{2}}{4}&\frac{\sqrt{2}}{4}\\
\frac{\sqrt{2}}{4}&-\frac{\sqrt{2}}{4}&-\frac{\sqrt{2}}{4}&-\frac{\sqrt{2}}{4}&-\frac{\sqrt{2}}{4}&\frac{\sqrt{2}}{4}
&\frac{\sqrt{2}}{4}&\frac{\sqrt{2}}{4}\\
\frac{1}{2}&0&0&0&0&-\frac{1}{2}
 &-\frac{1}{2}&\frac{1}{2}\\
  0&-\frac{1}{2}&-\frac{1}{2}&\frac{1}{2}&\frac{1}{2}&0
&0&0\\
\frac{\sqrt{2}}{2}&0&0&0&0&0
&0&-\frac{\sqrt{2}}{2}\\
  0&-\frac{\sqrt{2}}{2}&\frac{\sqrt{2}}{2}&0&0&0
&0&0\\
  0&0&0&-\frac{\sqrt{2}}{2}&\frac{\sqrt{2}}{2}&0
&0&0\\
  0&0&0&0&0&-\frac{\sqrt{2}}{2}
&\frac{\sqrt{2}}{2}&0&\end{bmatrix}$$

In [ ]:
W0 = dwt_haar(np.array([1,0,0,0]))
W1 = dwt_haar(np.array([0,1,0,0]))
W2 = dwt_haar(np.array([0,0,1,0]))
W3 = dwt_haar(np.array([0,0,0,1]))

W_dwt = np.array([W0,W1,W2,W3]).T
np.linalg.norm(W_dwt- matrizDWT(4,1))

